<a href="https://colab.research.google.com/github/benedettacandelori/Monkeys_go_stop_trials/blob/main/transformers_2D_causality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import scipy.io
import os
import math
import torch

In [ ]:
os.chdir('/content/drive/MyDrive')
os.getcwd() 

'/content/drive/MyDrive'

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
%pip install pytorch_lightning einops seaborn --quiet
!pip install positional-encodings[pytorch,tensorflow] --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.3 MB/s eta 0:00:00


In [ ]:
import math
import torch
import seaborn as sns
import matplotlib.pyplot as plt
from torch import nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import pytorch_lightning as pylight
from einops.layers.torch import Rearrange
from einops import reduce, rearrange, repeat
from positional_encodings.torch_encodings import PositionalEncoding1D, PositionalEncoding2D, PositionalEncoding3D, Summer
from torchmetrics.classification import Accuracy, F1Score, MulticlassAccuracy,BinaryAccuracy

# Dataset

### Dataset creation

In [ ]:
class Monkeys_dataset(Dataset):
    """Monkeys dataset"""

    def __init__(self, mat_file_labels, mat_file_x, root_dir, start_idx = 800, end_idx = 1200):
        """
        Args:
            mat_file (string): Path to the .mat files.
            root_dir (string): Directory with all the data.
        """
        self.classes = scipy.io.loadmat(mat_file_labels)['nostop']
        self.data = scipy.io.loadmat(mat_file_x)['matrix']
        self.root_dir = root_dir
        self.start_idx = start_idx
        self.end_idx = end_idx

    def __len__(self):
        return len(self.classes)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()


        x = self.data[:,idx,self.start_idx:self.end_idx]
        x = torch.tensor(x).float().squeeze(0)#.unsqueeze(1) 
        label = int(self.classes[idx,1])
      
        #sample = {'matrix': x, 'class': label}
        sample = (x, label)

        return sample

In [ ]:
start= 200
end = 700

In [ ]:
dataset = Monkeys_dataset(mat_file_labels = 'Stop_Task_Collab_2023/BHV_Go.mat', mat_file_x = 'Stop_Task_Collab_2023/activity_movement_onset_Go.mat',
                                    root_dir='Stop_Task_Collab_2023/', start_idx = start, end_idx = end)


In [ ]:
print(dataset[0][0].shape)
print(dataset[0][1])

torch.Size([105, 500])
2


### Data loader

In [ ]:
# We split the dataset in training and test portions
monkey_train, monkey_val, monkey_test = torch.utils.data.random_split(dataset, [0.7, 0.2, 0.1])

In [ ]:
batch_size = 8

In [ ]:
# Loaders create mini-batches of images from the original dataset
loader_train = torch.utils.data.DataLoader(monkey_train, batch_size=batch_size, shuffle=True,)
loader_val = torch.utils.data.DataLoader(monkey_val, batch_size=batch_size, shuffle=False)
loader_test = torch.utils.data.DataLoader(monkey_test, batch_size=batch_size, shuffle=False)

# Transformer architecture

In [ ]:
class SelfMHA(nn.Module):
  def __init__(self, d_input, d_value, d_query, d_output, n_heads=8, dropout=0.0):
      super().__init__()
      self.n_heads = n_heads
      self.query = nn.Linear(d_input, d_query * n_heads)
      self.keys = nn.Linear(d_input, d_query * n_heads)
      self.values = nn.Linear(d_input, d_value * n_heads)
      self.output = nn.Linear(d_value * n_heads, d_output)
      self.drop = nn.Dropout(dropout)

  def forward(self, x):
      Q = rearrange(self.query(x), 'b seq (q heads) -> b heads seq q', heads = self.n_heads)
      #print('Q-->',Q.shape)
      K = rearrange(self.keys(x), 'b seq (q heads) -> b heads seq q', heads = self.n_heads)
      #print('K-->',K.shape)
      V = rearrange(self.values(x), 'b seq (v heads) -> b heads seq v', heads = self.n_heads)
      #print('V-->',V.shape)
      A = torch.einsum('b h s q, b h t q -> b h s t', [Q, K]) / math.sqrt(Q.shape[-1])
      A = torch.softmax(A, dim=3)
      #print('A-->',A.shape)

      O = torch.einsum('b h s t, b h t v -> b h s v', [A, V])
      O = rearrange(O, 'b heads seq v -> b seq (heads v)')
      return self.drop(self.output(O)), A

In [ ]:
class TransformerBlock(nn.Module):
  def __init__(self, d_input, d_hidden, num_heads = 5):
    super().__init__()
    #self.sa = nn.MultiheadAttention(embed_dim=d_input, num_heads=num_heads, batch_first=True)
    self.sa = SelfMHA(d_input, d_hidden, d_hidden, d_input, n_heads=num_heads, dropout=0.)
    self.ln1 = nn.LayerNorm(d_input)
    self.ln2 = nn.LayerNorm(d_input)
    self.dense = nn.Sequential(
        nn.Linear(d_input, d_hidden),
        nn.GELU(),
        nn.Linear(d_hidden, d_input)
    )

  def forward(self, x):
    x = self.ln1(x)
    x_t, a = self.sa(x)
    x = x_t + x
    x = self.dense(self.ln2(x)) + x
    return x, a

In [ ]:
class TransformerEncoder(nn.Module):

  def __init__(self,d_input, d_hidden,num_heads=5, num_layers = 3, **block_args):
    super().__init__()
    self.layers = nn.ModuleList([TransformerBlock(d_input, d_hidden,num_heads) for _ in range(num_layers)])

  def forward(self, x):
    for l in self.layers:
      x, att = l(x)
    return x, att

  def get_attention_maps(self, x):
    attention_maps = []
    for l in self.layers:
        _, attn_map = l.sa(x)
        attention_maps.append(attn_map)
        x, att = l(x)
    return attention_maps

In [ ]:
from torch.functional import align_tensors
class Transformer(pylight.LightningModule):

  def __init__(self, d_space = 105,  d_time = 400, num_heads = 2, num_layers = 6, n_classes=2, 
               hidden_dimension=128, dropout = 0.1, learning_rate = 0.00001, positional_encoding = True):
    super().__init__()
    
    self.time = d_time
    self.add_pe = positional_encoding
    self.pos_enc = PositionalEncoding1D(d_time)
    self.transformer_space = TransformerEncoder(d_space, d_space, num_heads, num_layers)
    self.transformer_time = TransformerEncoder(d_time, d_time, num_heads, num_layers)

    self.classifier = nn.Linear(d_space, n_classes)
    self.acc = MulticlassAccuracy(num_classes=n_classes)
    self.learning_rate = learning_rate
    self.dropout = nn.Dropout(dropout)

  def forward(self, x): 
    if  self.add_pe:
      x = x + self.pos_enc(x)   
    x, a_s = self.transformer_time(x)
    x = rearrange(x, 'b s t -> b t s')
    x, a_t = self.transformer_space(x)
    x = self.dropout(x)
    #x = reduce(x, 'b time out -> b out', reduction='max')
    return self.classifier(x), a_s, a_t

  @torch.no_grad()
  def get_attention_map(self, x):
      """Function for extracting the attention matrices of the whole Transformer for a single batch.
      Input arguments same as the forward pass.
      """
      attention_maps_space = self.transformer_time.get_attention_maps(x)
      x = rearrange(x, 'b s t -> b t s')
      attention_maps_time = self.transformer_space.get_attention_maps(x)
      
      return attention_maps_space, attention_maps_time

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr = self.learning_rate) 
 
  def training_step(self, batch, batch_idx):
    xb, yb = batch
    yb = yb - 1
    y_pred, a_s, a_t = self(xb)
    
    yb = yb.reshape(3,1).repeat(1,500)
 
    loss = [F.cross_entropy(y_pred[:,t,:], yb[:,t].long()) for t in range(self.time)]
    acc = [self.acc(y_pred[:,t,:], yb[:,t].long()) for t in range(self.time)]

    self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
    self.log("train_acc", acc, on_step=False, on_epoch=True, prog_bar=True)
    return loss

  def validation_step(self, batch, batch_idx):
    xb, yb = batch
    yb = yb - 1
    
    y_pred, a_s, a_t = self(xb)


    yb = yb.reshape(3,1).repeat(1,500)
 
    loss = [F.cross_entropy(y_pred[:,t,:], yb[:,t].long()) for t in range(self.time)]
    acc = [self.acc(y_pred[:,t,:], yb[:,t].long()) for t in range(self.time)]


    self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
    self.log("val_acc", acc, on_step=False, on_epoch=True, prog_bar=True)
    return loss

  def test_step(self, batch, batch_idx):
    xb, yb = batch
    yb = yb - 1
    y_pred, a_s, a_t = self(xb)


    yb = yb.reshape(3,1).repeat(1,500)
    acc = [self.acc(y_pred[:,t,:], yb[:,t].long()) for t in range(self.time)]


    self.acc(y_pred, yb)
    self.log("test_acc", self.acc, on_step=False, on_epoch=True, prog_bar=True)
    return self.acc

In [ ]:
for xb, yb in loader_train:
  break
print(xb.shape)
print(yb.shape)

torch.Size([8, 105, 500])
torch.Size([8])


In [ ]:
#sanity check
y_pred, a_s, a_t= Transformer(d_time = 500, num_layers = 10, num_heads = 1)(xb)
#print(a_s.shape)
#print(a_t.shape)

# si ha predizione per ogni istante temporale
print(y_pred.shape)  # shape [b, T, n_classes]


torch.Size([8, 500, 2])


In [ ]:
T = 500
# reshape y_true: from [b] to [b, T] 
# per avere una classificazione per ogni istante temporale
yb_new = yb - 1
print(yb_new)
yb_new = yb_new.reshape(batch_size,1).repeat(1,T)
print(yb_new.shape)

print(yb_new)

tensor([0, 0, 1, 1, 1, 1, 1, 0])
torch.Size([8, 500])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0]])


In [ ]:
# Calcolo loss e accuracy per ogni istante temporale 
loss = [F.cross_entropy(y_pred[:,t,:], yb_new[:,t].long()).item() for t in range(T)]
accuracy_metric = MulticlassAccuracy(num_classes=2)
acc = [accuracy_metric(y_pred[:,t,:], yb_new[:,t].long()).item() for t in range(T)]
print(len(loss))
print(len(acc))

500
500


In [ ]:
model = Transformer(d_time  = 500, num_heads = 1, num_layers = 15, dropout = 0.3, learning_rate = 0.00005)
trainer = pylight.Trainer(max_epochs=30, accelerator='gpu', devices=1,default_root_dir="/content/drive/MyDrive/")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, loader_train, loader_val)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type                 | Params
-----------------------------------------------------------
0 | pos_enc           | PositionalEncoding1D | 0     
1 | transformer_space | TransformerEncoder   | 1.0 M 
2 | transformer_time  | TransformerEncoder   | 22.6 M
3 | classifier        | Linear               | 212   
4 | acc               | MulticlassAccuracy   | 0     
5 | dropout           | Dropout              | 0     
-----------------------------------------------------------
23.6 M    Trainable params
0         Non-trainable params
23.6 M    Total params
94.333    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

ValueError: ignored